In [ ]:
import pandas as pd
import csv
import numpy as np

import requests
from io import StringIO
from IPython.display import display
from random import shuffle
from itertools import combinations
import time


Task 2 Part 2

*  Concatenate the values in each record into one single string.
*  Change all alphabetical characters into lowercase.

*  Convert multiple spaces to one.
*  Combine the records from both tables into one big list.
*  Use the functions in the tutorials from lab 5 to compute the shingles, the
  minhash signature and the similarity.
* Extract the top 2224
 candidates from the LSH algorithm, compare them to the actual mappings in the file DBLP-ACM_perfectMapping.csv and compute the precision of the method.
*   Record the running time of the method.
*   Compare the precision and the running time in Parts 1 and 2.


In [ ]:
#Timing method
start_time_part2 = time.time()

# Load the datasets
df_ACM = pd.read_csv('ACM.csv', header=0, quotechar='"', sep=",", na_values=['na', '-', '.', ''])
df_DBLP2 = pd.read_csv('DBLP2.csv', header=0, quotechar='"', sep=",", encoding="ISO-8859-1", na_values=['na', '-', '.', ''])

#Checking datasete
head_ACM = df_ACM.head()
head_DBLP2 = df_DBLP2.head()

print("Head of ACM dataset:")
print(head_ACM)

print("\nHead of DBLP2 dataset:")
print(head_DBLP2)

Head of ACM dataset:
       id                                              title  \
0  304586  The WASA2 object-oriented workflow management ...   
1  304587  A user-centered interface for querying distrib...   
2  304589  World Wide Database-integrating the Web, CORBA...   
3  304590           XML-based information mediation with MIX   
4  304582  The CCUBE constraint object-oriented database ...   

                                             authors  \
0                    Gottfried Vossen, Mathias Weske   
1                  Isabel F. Cruz, Kimberly M. James   
2  Athman Bouguettaya, Boualem Benatallah, Lily H...   
3  Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...   
4  Alexander Brodsky, Victor E. Segal, Jia Chen, ...   

                                            venue  year  
0  International Conference on Management of Data  1999  
1  International Conference on Management of Data  1999  
2  International Conference on Management of Data  1999  
3  International Conferen

In [ ]:
#Checking for missing values
missing_values_ACM = df_ACM.isna().sum()
missing_values_DBLP2 = df_DBLP2.isna().sum()

print("Missing values in ACM dataset:")
print(missing_values_ACM)

print("\nMissing values in DBLP2 dataset:")
print(missing_values_DBLP2)

#Dropping missing values before combining records into single string
df_ACM.dropna(inplace=True)
df_DBLP2.dropna(inplace=True)

# Concatenate values in each record into one single string
df_ACM['combined'] = df_ACM.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_DBLP2['combined'] = df_DBLP2.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

Missing values in ACM dataset:
id          0
title       0
authors    14
venue       0
year        0
dtype: int64

Missing values in DBLP2 dataset:
id         0
title      0
authors    0
venue      0
year       0
dtype: int64


In [ ]:
#Change all alphabetical characters into lowercase
df_ACM['combined'] = df_ACM['combined'].str.lower()
df_DBLP2['combined'] = df_DBLP2['combined'].str.lower()

In [ ]:
#Convert multiple spaces to one.
df_ACM['combined'] = df_ACM['combined'].str.replace('\s+', ' ', regex=True)
df_DBLP2['combined'] = df_DBLP2['combined'].str.replace('\s+', ' ', regex=True)

In [ ]:
#Combine the records from both tables into one big list
combined_data = pd.concat([df_ACM['combined'], df_DBLP2['combined']], ignore_index=True)

#Checking the count of records and the df
count_combined = combined_data.count()

print("Count of records in combined dataset:")
print(count_combined)

count_combined_df = combined_data.to_frame()
print("\nCombined dataset:")
print(count_combined_df)

#Put data into a frame
combined_data = combined_data.to_frame()
combined_data.head()

Count of records in combined dataset:
4896

Combined dataset:
                                               combined
0     304586 the wasa2 object-oriented workflow mana...
1     304587 a user-centered interface for querying ...
2     304589 world wide database-integrating the web...
3     304590 xml-based information mediation with mi...
4     304582 the ccube constraint object-oriented da...
...                                                 ...
4891  journals/tods/karpsp03 a simple algorithm for ...
4892  conf/vldb/limwv03 sash: a self-adaptive histog...
4893  journals/tods/chakrabartikmp02 locally adaptiv...
4894  journals/sigmod/snodgrass01 chair's message ri...
4895  conf/vldb/lim01 indexing and querying xml data...

[4896 rows x 1 columns]


,combined
0,304586 the wasa2 object-oriented workflow mana...
1,304587 a user-centered interface for querying ...
2,304589 world wide database-integrating the web...
3,304590 xml-based information mediation with mi...
4,304582 the ccube constraint object-oriented da...


In [ ]:
#For step 6 the ID's will be listed.
combined_data_with_id = list(zip(df_ACM['id'], df_ACM['combined'])) + list(zip(df_DBLP2['id'], df_DBLP2['combined']))
combined_data_with_id[:5]

[(304586,
  '304586 the wasa2 object-oriented workflow management system gottfried vossen, mathias weske international conference on management of data 1999'),
 (304587,
  '304587 a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly m. james international conference on management of data 1999'),
 (304589,
  '304589 world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani international conference on management of data 1999'),
 (304590,
  '304590 xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou, pavel velikhov, vincent chu international conference on management of data 1999'),
 (304582,
  '304582 the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo international conference on management of data 

Compute the shingles, the minhash signature and the similarity.

In [ ]:
#Shingles
def shingle(text, k=5):
    return {text[i:i + k] for i in range(len(text) - k + 1)}

shingled_list = [shingle(text) for _, text in combined_data_with_id]
print(shingled_list[:5])

[{'em go', 'natio', 'ottfr', 'nal c', 'asa2 ', 'nt of', ' the ', 'manag', 'iente', 'athia', 'en, m', 'confe', ' of d', ' data', 'anage', '4586 ', 'tem g', 'n, ma', 'm got', 'ce on', ' mana', 'al co', 'e on ', 'ment ', 'ried ', '04586', 'e int', 'a 199', 'ssen,', 'ageme', 't sys', 'ow ma', 'erenc', 'nagem', ' wasa', 'ional', 'l con', 'on ma', 'feren', 'tiona', 'eske ', 'orien', 'kflow', ' gott', ' wesk', ' 1999', 'onal ', 'of da', 'ata 1', 'd wor', 'rnati', 'ntern', 'f dat', ' math', 'ke in', '-orie', '30458', 'ect-o', ' inte', 'low m', 'as we', 'ation', 'thias', 'stem ', 'e was', 'n man', 'ented', 'syste', 'ent s', 'ct-or', ' voss', ' conf', ', mat', 'fried', ' work', 'he wa', 'ttfri', 'hias ', 'orkfl', 'ernat', 'ted w', 'ject-', 'ystem', 'objec', 'nt sy', 'weske', 'terna', 't of ', 'ias w', 'rence', 'workf', 'tfrie', 'gottf', '2 obj', 'd vos', 'nted ', 'gemen', 'nfere', '586 t', 'rkflo', 'ossen', 'mathi', 'w man', 'ske i', 's wes', 'inter', 'onfer', 'ence ', '6 the', 'ied v', 'bject',

In [ ]:
#Function for vocab
def build_vocab(shingle_sets: list)->dict:
    full_set = {item for set_ in shingle_sets for item in set_}
    vocab = {}
    for i, shingle in enumerate(list(full_set)):
        vocab[shingle] = i
    return vocab
#Function for one_hot encoded vector from shingles list
def one_hot(shingles: set, vocab: dict):
    vec = np.zeros(len(vocab))
    for shingle in shingles:
        idx = vocab[shingle]
        vec[idx] = 1
    return vec

vocab = build_vocab(shingled_list)
document_shingle_matrix = [one_hot(s, vocab) for s in shingled_list]

In [ ]:
#MinHashing
def get_minhash_arr(num_hashes:int,vocab:dict):
    length = len(vocab.keys())
    arr = np.zeros((num_hashes,length))
    for i in range(num_hashes):
        permutation = np.random.permutation(len(vocab.keys())) + 1
        arr[i,:] = permutation.copy()
    return arr.astype(int)

def get_signature(minhash:np.ndarray, vector:np.ndarray):
    idx = np.nonzero(vector)[0].tolist()
    shingles = minhash[:,idx]
    signature = np.min(shingles,axis=1)
    return signature

def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    return intersection_size / union_size if union_size != 0 else 0.0

def compute_signature_similarity(signature_1, signature_2):
    if signature_1.shape != signature_2.shape:
        raise ValueError("Both signature matrices must have the same shape.")
    agreement_count = np.sum(signature_1 == signature_2)
    similarity = agreement_count / signature_2.shape[0]

    return similarity

In [ ]:
# MinHash Array
num_hashes = 500  # The number of hash functions to use for MinHash
minhash_array = get_minhash_arr(num_hashes, vocab)

# Compute MinHash Signatures for all documents
signatures = [get_signature(minhash_array, vec) for vec in document_shingle_matrix]

# Estimated Jaccard similarity using MinHash signatures
similarity_estimate = compute_signature_similarity(signatures[0], signatures[1])

# Actual Jaccard similarity between the original shingle sets
actual_similarity = jaccard_similarity(shingled_list[0], shingled_list[1])

print(f"Estimated Jaccard Similarity: {similarity_estimate}")
print(f"Actual Jaccard Similarity: {actual_similarity}")

Estimated Jaccard Similarity: 0.192
Actual Jaccard Similarity: 0.20502092050209206


In [ ]:
#LSH function (Lab):
class LSH:
    buckets = []
    counter = 0

    def __init__(self, b: int):
        self.b = b
        for i in range(b):
            self.buckets.append({})

    def make_subvecs(self, signature: np.ndarray) -> np.ndarray:
        l = len(signature)
        assert l % self.b == 0
        r = int(l / self.b)
        subvecs = []
        for i in range(0, l, r):
            subvecs.append(signature[i:i+r])
        return np.stack(subvecs)

    def add_hash(self, signature: np.ndarray):
        subvecs = self.make_subvecs(signature).astype(str)
        for i, subvec in enumerate(subvecs):
            subvec = ','.join(subvec)
            if subvec not in self.buckets[i].keys():
                self.buckets[i][subvec] = []
            self.buckets[i][subvec].append(self.counter)
        self.counter += 1

    def check_candidates(self) -> set:
        candidates = []
        for bucket_band in self.buckets:
            keys = bucket_band.keys()
            for bucket in keys:
                hits = bucket_band[bucket]
                if len(hits) > 1:
                    candidates.extend(combinations(hits, 2))
        return set(candidates)

In [ ]:
b = 50   #number of buckets
lsh = LSH(b)
for signature in signatures:
    lsh.add_hash(signature)
candidate_pairs = lsh.check_candidates()
# Get top 2224 candidates
top_candidates = list(candidate_pairs)[:2224]
print(top_candidates[:10])


[(1905, 3896), (2481, 3284), (86, 4358), (2195, 4336), (657, 921), (191, 4667), (2591, 3262), (1237, 2170), (2494, 3082), (649, 926)]


In [ ]:
df_PerfectMapping = pd.read_csv('DBLP-ACM_perfectMapping.csv', header=0, quotechar='"', sep=",", na_values=['na', '-', '.', ''])
df_PerfectMapping.head()

,idDBLP,idACM
0,conf/sigmod/SlivinskasJS01,375678
1,conf/sigmod/ChaudhuriDN01,375694
2,conf/sigmod/RinfretOO01,375669
3,conf/sigmod/BreunigKKS01,375672
4,conf/sigmod/JagadishJOT01,375687


In [ ]:
actual_mappings = set(tuple(rec) for rec in df_PerfectMapping[['idACM', 'idDBLP']].to_records(index=False))
# Determine where ACM ends and DBLP begins.
split_index = len(df_ACM)

# Filter the candidates to ensure that one ID is from ACM and the other is from DBLP2
filtered_candidates = [(i, j) for i, j in candidate_pairs if i < split_index and j >= split_index]

# Get the top 2224 candidates from the filtered list
top_candidates_filtered = filtered_candidates[:2224]

#Construct top_candidates_ids_adjusted using the filtered candidate pairs
top_candidates_ids_adjusted = {(combined_data_with_id[i][0], combined_data_with_id[j][0]) for i, j in top_candidates_filtered}

# Compute precision using the adjusted top_candidates_ids
correct_candidates_adjusted = actual_mappings.intersection(top_candidates_ids_adjusted)
precision_adjusted = len(correct_candidates_adjusted) / len(top_candidates_ids_adjusted)

print(f"Adjusted Precision: {precision_adjusted:.4f}")
print("Correctly identified pairs:", correct_candidates_adjusted)
print(len(correct_candidates_adjusted))

# Ending time
end_time_part2 = time.time()

Adjusted Precision: 0.9888
Correctly identified pairs: {(233358, 'conf/sigmod/ChristophidesCM96'), (233311, 'conf/sigmod/SrikantA96'), (672187, 'conf/vldb/MorishimaKKT01'), (673484, 'conf/vldb/GardarinGT96'), (335495, 'conf/sigmod/WangSSZZW00'), (362136, 'journals/sigmod/Bussche00'), (671841, 'conf/vldb/AltinelF00'), (253400, 'conf/sigmod/GeneserethKD97'), (872842, 'conf/sigmod/ShenNSHS03'), (262766, 'journals/sigmod/HadzilacosT97'), (304219, 'conf/sigmod/Keim99'), (673169, 'conf/vldb/GardarinGT95'), (362140, 'journals/sigmod/LeeC00'), (673307, 'conf/vldb/MogiK95'), (671015, 'conf/vldb/LeeLWS97'), (212022, 'journals/sigmod/Qian95a'), (671183, 'conf/vldb/RastogiS98'), (245895, 'journals/sigmod/KennedyMB96'), (671005, 'conf/vldb/CiacciaPZ97'), (362101, 'journals/sigmod/ClementiniF00'), (765532, 'journals/vldb/KraissW98'), (640997, 'journals/sigmod/MarchiLPT03'), (672199, 'conf/vldb/CareyKRLABFLOPQRSSUVWY01'), (945733, 'journals/sigmod/ArenasKKKMM03'), (673490, 'conf/vldb/LakshmananSS96')

In [ ]:
total_time = end_time_part2 - start_time_part2
print(f"Total running time: {total_time:.2f} seconds")

Total running time: 14.73 seconds
